## 冰冻湖问题

### 值迭代

In [1]:
import gym
import numpy as np

In [2]:
# 创建环境
env = gym.make('FrozenLake-v0')

[2019-06-21 10:24:21,595] Making new env: FrozenLake-v0
C:\Users\lovemoon\Anaconda3\envs\tensorflow-gpu\lib\site-packages\gym\envs\registration.py:18: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
env.render()

SFFF
FHFH
FFFH
HFFG



In [5]:
# 探索环境
print(env.observation_space.n)
print(env.observation_space)

16
Discrete(16)


准备定义一个 **value_iteration()**函数来返回最优值函数（值表）

In [6]:
## 首先初始化值表，所有状态的值均为0，并设置迭代次数 
value_table = np.zeros(env.observation_space.n)
no_of_iterations = 100000

In [7]:
print(value_table)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [4]:
def value_iteration(env, gamma = 1.0):
    
    # initialize value table with zeros
    value_table = np.zeros(env.observation_space.n)
    
    # set number of iterations and threshold
    no_of_iterations = 100000
    threshold = 1e-20
    
    for i in range(no_of_iterations):
        
        # On each iteration, copy the value table to the updated_value_table
        updated_value_table = np.copy(value_table) 
        
        # Now we calculate Q Value for each actions in the state 
        # and update the value of a state with maximum Q value
        
        for state in range(env.observation_space.n):
            Q_value = []
            for action in range(env.action_space.n):
                next_states_rewards = []
                for next_sr in env.P[state][action]: 
                    trans_prob, next_state, reward_prob, _ = next_sr 
                    next_states_rewards.append((trans_prob * (reward_prob + gamma * updated_value_table[next_state]))) 
                
                Q_value.append(np.sum(next_states_rewards))
                
            value_table[state] = max(Q_value) 
            
        # we will check whether we have reached the convergence i.e whether the difference 
        # between our value table and updated value table is very small. But how do we know it is very
        # small? We set some threshold and then we will see if the difference is less
        # than our threshold, if it is less, we break the loop and return the value function as optimal
        # value function
        
        if (np.sum(np.fabs(updated_value_table - value_table)) <= threshold):
             print ('Value-iteration converged at iteration# %d.' %(i+1))
             break
    
    return value_table
    

我们已经得到了**optimal_value_function**后，需要从其中提取最优策略，先根据最优值行为计算Q值，并取对每个状态具有最大Q值的行为为最优策略。

In [5]:
def extract_policy(value_table, gamma = 1.0):
 
    # initialize the policy with zeros
    policy = np.zeros(env.observation_space.n) 
    
    
    for state in range(env.observation_space.n):
        
        # initialize the Q table for a state
        Q_table = np.zeros(env.action_space.n)
        
        # compute Q value for all ations in the state
        for action in range(env.action_space.n):
            for next_sr in env.P[state][action]: 
                trans_prob, next_state, reward_prob, _ = next_sr 
                Q_table[action] += (trans_prob * (reward_prob + gamma * value_table[next_state]))
        
        # select the action which has maximum Q value as an optimal action of the state
        policy[state] = np.argmax(Q_table)
    
    return policy

In [6]:
optimal_value_function = value_iteration(env=env,gamma=1.0)

Value-iteration converged at iteration# 1373.


In [7]:
optimal_policy = extract_policy(optimal_value_function, gamma=1.0)

In [8]:
print(optimal_policy)

[ 0.  3.  3.  3.  0.  0.  0.  0.  3.  1.  0.  0.  0.  2.  1.  0.]
